In [1]:
from langchain.document_loaders import SeleniumURLLoader
import os
from dotenv import load_dotenv
import openai
import logging
from langchain.document_loaders import Docx2txtLoader


load_dotenv(".env")
openai.api_key = os.environ.get("OPENAI_API_KEY")
# Create a logger
logging.basicConfig(filename='example.log',level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# Flow
# Extract key words from job description
# Remove irrelevant key ords that are not relatable to resume
# return key words and coresponding resume experience
# combine and write into cover letter, extract position title

# input job description link
# resume upload
# extra instructions

In [2]:
def chatbot_completition(system_message, user_message, temp=None):
    if temp is not None:
        response = openai.ChatCompletion.create(
            model = 'gpt-4',
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message},
            ],
            temperature = temp,
        )
    else:
        response = openai.ChatCompletion.create(
            model = 'gpt-4',
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message},
            ],
        )    
    
    prompt, reply = response['usage']['prompt_tokens'], response['usage']['completion_tokens']
    cost = (0.03/1000)*prompt + (0.06/1000)*reply
    ## calculate api costs and log it
    logger.info(f"Prompt tokens: {prompt}, Completion tokens: {reply}, Cost: {cost}")

    return response["choices"][0]["message"]["content"]

In [3]:
def extract_key_words_and_resp(text):
    """
    Extracts key words from a text
    """
    SYS = """You are an expert career coach and hiring manager, responsible for extracting key aspects from a job posting. Your task includes identifying essential responsibilities, skills, and keywords. Do not introduce any additional information.

    The following examples should be excluded from your extraction as they do not represent intangible or technical skills, competencies, or job responsibilities:

    Location specific information (e.g., "Melbourne")
    Citizenship requirements (e.g., "Australian Citizen/Permanent Resident")
    Job perks and incentives (e.g., "Uncapped commission", "Weekly bonuses and commissions")
    Specific company details or affiliations (e.g., "Potential for a part of the global company", "Global blue-chip client", "American Express")
    Physical mobility and travel needs (e.g., "Having a current valid driver's license and a reliable vehicle", "Regional/interstate travel")
    Provided equipment (e.g., "Tools of the trade (tablet and phone) provided")
    Specific experience in certain industry other than sales or account management  (e.g., "3+ years sales experience within the international freight industry")
    Employment type (e.g., "Full time maximum term contract")
    Personal and career development opportunities (e.g., "Career change", "Entry level sales", "Development program", "Job training", "Mentorship", "Personalised coaching", "High achievers", "Future leaders")
    Superannuation details (e.g., "Super")
    Specific initiatives or programs (e.g., "Microcredentials initiatives", "Learning and Leadership Enterprise (LLE)")
    Instructions to review documentation (e.g., "Review the Position Description")
    Application process details (e.g., "Responses to the Selection Criteria and Core Competencies", "Online responses to pre-screening employment questions", "Application process understanding")
    Job application preparation (e.g., "Resume creation", "Cover letter writing")
    Support offered to candidates (e.g., "Support for candidates requiring adjustments", "Business School support")
    Privacy concerns (e.g., "Privacy of personal information")
    
    ie DONT NOT EXTRACT things that are not intagible skills/competencies or techincal skills or competencies.Your task is to focus on the technical and intangible skills, competencies, and job responsibilities that are described in the job posting. 
            
    Respond using markdown"""
    
    USER = """Given the following job posting, extract keywords that would be relevant to include in the experiences of a resume. Think this through step by step. Look for the relevant detail and keywords from the job posting:
                \n\n"""
    words = chatbot_completition(SYS, USER + text)
    
    
    return words



In [4]:
def filter_key_words(key_words, resume):
    """
    Extracts key words from a text
    """
    SYS = """You are an expert career coach and hiring manager, responsible for tailoring a resume perfectly for a job posting. Your task is to filter the given key words from a job posting  according to the resume you are provided with. Only remove key words that cannot be tailored to the resume or that the resume experience could not be tailored ot those key words.  Do not introduce any additional information.
            
    Format as a single list of key words. Respond using markdown"""
    
    USER = f"""Given the following keywords, extract those that would be relevant to include with the experiences of the provided resume. Think this through step by step.
    
    Key words to filter from job posting: {key_words}
    Resume: {resume}
                \n\n"""
    output = chatbot_completition(SYS, USER)
    
    
    return output

In [5]:
def extract_resume_exp(key_words, resume):
    """
    Extracts key words from a text
    """
    SYS = """You are an expert career coach and hiring manager, responsible for tailoring a resume perfectly for a job posting. Your task is to extract the experience from the given resume you are provided with that is related to the key words and job position provided. Only extract experience that can be tailored to the key words.  Do not introduce any additional information.
            
    Do not reduce the detail of the extracted experience. Respond using markdown"""
    
    USER = f"""Given the following resume, extract the experience that would be relevant to include according to the key words provided from the job posting. Think this through step by step.
    
    Key words from job posting: {key_words}
    Resume: {resume}
                \n\n"""
    output = chatbot_completition(SYS, USER)
    
    
    return output

In [6]:
def write_coverletter(job_desc,key_words, experience, personal_dets, instructions):
    """
    Extracts key words from a text
    """
    SYS = f"""You are an expert hiring manager and cover letter writer, responsible for tailoring a cover letter perfectly for a job posting.{instructions}  Respond using markdown"""
    
    USER = f""" Given the below job description, relevant key words and resume experiences from the candidate, complete the template. 
            Job description:{job_desc}
            Key words from job posting: {key_words}
            Relevant experiences: {experience}
            Personal details: {personal_dets}

            Template:
            Dear [Hiring Manager's Name],

            I am writing to express my interest in the [Job Title] position at [Company Name] as advertised. I am a [Your Profession] with a background in [Relevant Fields or Industries] and believe that my unique skills and experiences make me a strong candidate for this position.
            In my current/previous role as [Your Current/Previous Job Title] at [Your Current/Previous Company], I [describe a key accomplishment or responsibility that showcases relevant skills]. For example, I [specific example of a time you used a particular skill or demonstrated a particular achievement]. I believe these experiences align with the requirements you are seeking for the [Job Title] role.
            What interests me about [Company Name] is [specific details about the company that have drawn your interest]. Your commitment to [specific aspect of the company's work or culture] particularly resonates with my own professional values.
            I am particularly adept at [another skill or two that is mentioned in the job posting], as demonstrated by [another specific example]. My [mention a personal quality or qualities] will allow me to [explain how you would use your personal qualities to succeed in the job].
            Thank you for considering my application. I am excited about the possibility of contributing to [Company Name] and look forward to the opportunity to further discuss how my background, skills, and experience can benefit your team.
            Please feel free to contact me at your earliest convenience to arrange a time to discuss how I can make a meaningful contribution to your organization.

            Sincerely,

            [Your Full Name]
            """


    output = chatbot_completition(SYS, USER)
    
    
    return output

In [8]:
#input variables
# job_urls = [
    # "https://www.seek.com.au/job/68801342?type=standard#sol=e17d1421904e325347205c9167400bdd5c7a5d27",
#     "https://www.seek.com.au/job/68982589?type=standout#sol=a9f3fa61762231385bd75a9da80a43d0600ea29c",
#     "https://www.seek.com.au/job/68954499?type=standard#sol=107f2b3f1e1d99f08406205d42f254516cd97019",
    # "https://www.seek.com.au/job/68766133?type=standout#sol=fcae9827342ce28d57852f76458bc407d827f490",
#     "https://www.seek.com.au/job/68912013?type=standout#sol=f52041b683b7c944c9878364ef8e73962c495328",
    # "https://www.seek.com.au/job/68937311?type=standout#sol=19b248d348659b4a305783437236a4a6420f82ab"
# ]
job_urls = 'https://www.seek.com.au/job/68922342?type=standout#sol=3947ab745533e8751f51dfd6055c802d86c49572'
resume_file = 'Patrick_Merola_Resume.docx'
personal_details = "Patrick Merola"
# personal_details = "Eleni Halkias"
# receipient = "HR Manager"
extra_instructions = " "

In [9]:
# Flow
# Extract key words from job description
# Remove irrelevant key ords that are not relatable to resume
# return key words and coresponding resume experience
# combine and write into cover letter, extract position title

# input job description link
# resume upload
# extra instructions

# url_loader = SeleniumURLLoader(urls=job_urls)
from langchain.document_loaders import WebBaseLoader
url_loader = WebBaseLoader(job_urls)
postings = url_loader.load()
# postings = url_loader.load()
# job_descr = [i.page_content for i in postings]

job_descr = postings[0].page_content

resume_loader = Docx2txtLoader(resume_file)
resume = resume_loader.load()[0].page_content


In [10]:
key_words = extract_key_words_and_resp(job_descr)
print(key_words)

**Job Responsibilities:**
- Responsible for the effective and efficient day-to-day operations of a range of projects
- Provide support for major initiatives, commercial and government tenders, national and international outreach and engagement, and partner development
- Provide administrative support for the implementation of projects, delivery of projects and initiatives, and support the processes and frameworks required to deliver on project and organisational objectives
- Support the development and submission of tenders and proposals
- Provide event coordination support for various events and learning programs
- Stakeholder engagement with a variety of internal and external stakeholders
- Support for strategic initiatives

**Skills and Competencies:**
- Excellent communication skills
- Excellent organization skills
- Ability to work in a fast-paced environment
- Ability to negotiate with a variety of stakeholders
- Project management
- Event coordination
- Stakeholder engagement
- 

In [ ]:
filtered_key_words = filter_key_words(key_words,resume)
print(filtered_key_words)

In [ ]:
relevant_exp = extract_resume_exp(filtered_key_words,resume)
print(relevant_exp)

In [ ]:
cover_letter = write_coverletter(job_descr, filtered_key_words,relevant_exp, personal_details, extra_instructions)

In [ ]:
with open('cover_letter.txt', 'w') as f:
    f.write(cover_letter)



In [57]:
from docx import Document

In [66]:
with open('cover_letter.txt', 'r') as f:
    cover_letter_text = f.read()

In [69]:
doc = Document()

file_path = "output.docx"
# Split the cover letter text into paragraphs by new lines
paragraphs = cover_letter_text.split('\n')

# Iterate through the paragraphs and add them to the document
for paragraph in paragraphs:
    if paragraph:  # Ignore empty lines
        doc.add_paragraph(paragraph)

# Save the document
doc.save(file_path)


In [5]:
from selenium import webdriver
job_url = 'https://www.seek.com.au/job/68922342?type=standout#sol=3947ab745533e8751f51dfd6055c802d86c49572'
def test_url_loading(job_url):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)

    try:
        driver.get(job_url)
        print(driver.title)  # Print the title of the page
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    finally:
        driver.quit()
test_url_loading(job_url)

Administrative Officer Job in Fitzroy, Melbourne VIC - SEEK


In [2]:
# def process_job_link(job_url):
#     # Logging the URL to check if it's correct
#     # logger.info(f"Processing job URL: {job_url}")

#     # Basic validation to check if it's a string and starts with "http"
#     if not isinstance(job_url, str) or not job_url.startswith("http"):
#         # logging.error(f"Invalid job URL: {job_url}")
#         return None

#     url_loader = SeleniumURLLoader(urls=job_url)
#     postings = url_loader.load()
#     # job_descr = [i.page_content for i in postings]
#     job_description = postings[0].page_content


#     return job_description

# process_job_link('https://www.indeed.com/viewjob?jk=1b5b5b5b5b5b5b5b&tk=1f9b5b5b5b5b5b5b&from=serp&vjs=3')


job_url = "https://www.seek.com.au/job/68922342?type=standout#sol=3947ab745533e8751f51dfd6055c802d86c49572"
url_loader = SeleniumURLLoader(urls=job_url)
postings = url_loader.load()
# job_descr = [i.page_content for i in postings]
job_description = postings[0].page_content


IndexError: list index out of range

In [15]:

postings

[Document(page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAdministrative Officer Job in Fitzroy, Melbourne VIC - SEEK\n\nAdministrative OfficerSkip to contentJobsCoursesBusinesses for saleVolunteeringSEEKJob searchProfileCareer adviceCompany reviewsSign inEmployer siteMenuJob searchProfileCareer adviceCompany reviewsSign inAustraliaAustraliaHong KongIndonesiaMalaysiaNew ZealandPhilippinesSingaporeThailandEmployer siteAll SEEK productsJobsCoursesBusinesses for saleVolunteeringJob searchProfileCareer adviceCompany reviewsEmailFacebookTwitterLinkedInAdministrative OfficerAustralian Catholic University Limited3.7·67 reviewsFitzroy, Melbourne VICAdministrative Assistants (Administration & Office Support)Other (Education & Training)Full time$78,048 – $84,489 plus 17% superPosted 13d agoMore jobs from this companyApply\u2060SaveKey Administrative Support role in Learning and Leadership Enterprise (LLE)17% super, salary packaging, Staff Rewards Program and generou

In [14]:
from bs4 import BeautifulSoup
import requests

url = "https://en.wikipedia.org/wiki/Python_(programming_language)"
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

text = soup.get_text()

[Document(page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAdministrative Officer Job in Fitzroy, Melbourne VIC - SEEK\n\nAdministrative OfficerSkip to contentJobsCoursesBusinesses for saleVolunteeringSEEKJob searchProfileCareer adviceCompany reviewsSign inEmployer siteMenuJob searchProfileCareer adviceCompany reviewsSign inAustraliaAustraliaHong KongIndonesiaMalaysiaNew ZealandPhilippinesSingaporeThailandEmployer siteAll SEEK productsJobsCoursesBusinesses for saleVolunteeringJob searchProfileCareer adviceCompany reviewsEmailFacebookTwitterLinkedInAdministrative OfficerAustralian Catholic University Limited3.7·67 reviewsFitzroy, Melbourne VICAdministrative Assistants (Administration & Office Support)Other (Education & Training)Full time$78,048 – $84,489 plus 17% superPosted 13d agoMore jobs from this companyApply\u2060SaveKey Administrative Support role in Learning and Leadership Enterprise (LLE)17% super, salary packaging, Staff Rewards Program and generou

Improve by creating template in microsoft word and generating with download link by populating the placeholders
Can process multiple jb descriptiosn and iterate over list of lists to create multiple cover letters
can select how many versions to generate, max 3
rewrite using gpt functions 